# Some Python Identities




### See https://github.com/steveholden/pydentities/

## len(x) ≡ x.\_\_len\_\_()

In [ ]:
class Fixed_size:
    def __len__(self):
        return 42

In [ ]:
fs = Fixed_size()
len(fs), fs.__len__()

## x.method(\*args, \*\*kw) ≡ (x.\_\_class\_\_).method(x, \*args, \*\*kw)

In [ ]:
class MyClass1:
    def method(self, positional, keyword="keyword"):
        print(self, positional, keyword)

In [ ]:
mc = MyClass1()
mc.method("hello", "world")

In [ ]:
mc.__class__

In [ ]:
mc.__class__.method

In [ ]:
mc.__class__.method(mc, "hello", "world")

In [ ]:
fs.__len__(), fs.__class__.__len__(fs), fs.__class__ is Fixed_size

## x in y ≡ y.\_\_contains\_\_(x)

In [ ]:
("BCD" in "ABCDEF", "ABCDEF".__contains__("BCD"))

In [ ]:
class SearchableListOfLists:
    def __init__(self, *args):
        self._values = args
    def __contains__(self, value):
        return any(value in thing for thing in self._values)

In [ ]:
sl = SearchableListOfLists([1, 2, 3], [4, 5, 6], [7, 8, 9])
[i in sl for i in range(11)]

## items[key] ≡ items.\_\_getitem\_\_(key)

In [ ]:
class MyContainer:
    def __init__(self, value):
        self.value = value
    def __getitem__(self, key):
        print("Key:", key)
        return 42

In [ ]:
mc = MyContainer(", world")
mc["hello"], mc["goodbye"]

In [ ]:
mc[1:]

In [ ]:
"abcdefghijklmnopqrstuvwxyz"[slice(1, None, None)]

In [ ]:
mc[20:1:]

In [ ]:
mc[20:1:-2]

In [ ]:
"abcdefghijklmnopqrstuvwxyz"[slice(20, 1, -2)]

## items[key] = value≡ items.\_\_setitem\_\_(key, value)

In [ ]:
class SettableItems:
    def __init__(self, v):
        self._list = []
        self.v = v
    def __setitem__(self, key, value):
        self._list.append((self.v, key, value))
    def dump(self):
        for item in self._list:
            print(item)

In [ ]:
s = SettableItems("@@")
s["hello"] = "World"
s["farewell"] = "Cruel world"
s.dump()

## f(\*args, \*\*kwargs) ≡ f.\_\_call\_\_(\*args, \*\*kwargs)

In [ ]:
s1 = SettableItems.__call__("££")
s1[1] = 2
s1[3] = 4
s1.dump()

## Therefore ...
## f(\*args, \*\*kwargs) ≡ f\_\_class\_\_.\_\_call\_\_(f, \*args, \*\*kwargs)

In [ ]:
def f(x, y):
    return x+y

f("Hello", "world")

In [ ]:
f.__call__("Hello", "world")

In [ ]:
f.__call__.__call__("Hello", "world")

In [ ]:
f.__call__.__call__.__call__("Hello", "world")

In [ ]:
f.__call__.__call__.__call__.__call__("Hello", "world")

In [ ]:
f.__class__, type(f)

In [ ]:
f.__class__.__call__(f, "Hello", "world")

In [ ]:
SettableItems.__class__

In [ ]:
s2 = type.__call__(SettableItems, "##")
s2[1] = "one"
s2[2] = "two"
s2.dump()

# And finally ...

## class X(base1, base2, ...): ... ≡ X = type('X', (base1, base2, ...), { definitions_dict})

In [ ]:
def m(self, other):
    return self is other

In [ ]:
T = type('T++', (object, ), {"is_same_as": m})
T

In [ ]:
t = T()
t

In [ ]:
t.is_same_as(None), t.is_same_as(t)

In [ ]:
class print_args(type):
    def __new__(cls, name, bases, name_space):
        print("class:", cls)
        print("Declaring class", name)
        print("Bases:", bases)
        print("Namespace:", name_space)
        return type.__new__(cls, name, bases, name_space)

In [ ]:
class A: pass

class B: pass

In [ ]:
print("Defining C")
class C(A, B, metaclass=print_args):
    CVar = "class attribute"
    def myMethod(self, a):
        return self.N, self.CVar
    def __init__(self, N):
        self.N = N

In [ ]:
print("Creating c")
c = C(42)
print("CVars:", C.CVar, c.CVar)
print("C:", C.__dict__)
print("c:", c.__dict__)

In [ ]:
c.CVar = "instance attribute"
print("CVars:", c.CVar, C.CVar)
print("c:", c.__dict__)